# Automatic Data Type Discovery

In [1]:
from scipy.io import savemat
import openml as oml
import pandas as pd
import numpy as np
import random

## 1. Heuristic Methods

In [2]:
def infer_feature_type(feature):
    """
    Infer data types for the given feature using simple logic
    Possible data types to infer: boolean, date, float, integer, string
    Feature that is not either a boolean, a date, a float or an integer, is classified as a string
    """
    data_type = ""
    types = ["date","float64","int64","string"]
    weights = [0,0,0,0] #Weights corresponding to the data types
    feature_len = len(feature)
    
    indices_number = int(0.1 * feature_len) #Number of different values to check in a feature
    indices = random.sample(range(0,feature_len), min(indices_number,feature_len)) #Array of random indices
    
    # If the feature only contains two different unique values, then infer it as boolean
    if len(pd.unique(feature)) == 2:
        data_type = "bool"
    else:
        for i in indices:
            try:
                if (len(feature[i]) <= 10 
                    and (((feature[i][2:3]=='-' or feature[i][2:3]=='/') 
                    and (feature[i][5:6]=='-' or feature[i][5:6]=='/')) 
                    or ((feature[i][4:5]=='-' or feature[i][4:5]=='/')
                    and (feature[i][7:8]=='-' or feature[i][7:8]=='/')))):
                    weights[0] += 1 #Date
                else:
                    weights[3] += 1 #String
            except (TypeError,ValueError,IndexError):
                try:
                    int(feature[i]) # numeric
                    if ('.' in str(feature[i])):
                        if isinstance(feature[i], np.float64):
                            if feature[i].is_integer():
                                weights[2] += 1 # Integer
                            else:
                                weights[1] += 1 # Float
                        else:
                            weights[1] += 1 # Float
                    else:
                        weights[2] += 1 #Integer
                except (TypeError,ValueError,IndexError):
                    weights[3] += 1 #String
#         print ("Date: {}, Float64: {}, Int64: {}, String: {}".format(weights[0],weights[1],weights[2],weights[3])) #For debugging purposes
        data_type = types[weights.index(max(weights))]
        
    return data_type

def discover_type_heuristic(data):
    """
    Automatically infer data types for each feature in the dataframe using simple logic
    ---
    input:

    data : numpy array or dataframe (numeric data needs to be 64 bit)
        
    """
#     df = pd.DataFrame(data)
#     print(df)
    result = []
    if isinstance(data, np.ndarray):
        # convert float32 to float64
        data = np.array(data, dtype='float64')
        df = pd.DataFrame(data)
    else:
        df = data

    for column in df.columns:        
        #print ("Trying to automatically infer the data type of the",column,"feature...") #For debugging purposes
        type_inferred = infer_feature_type(df[column])
        result.append(type_inferred)
        #print ("Result:",inferredType) #For debugging purposes
    return result

### 1.1 Get data from csv

In [3]:
df = pd.read_csv("datasets_csv/tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
discover_type_heuristic(df)

['float64', 'float64', 'bool', 'bool', 'string', 'bool', 'int64']

### 1.2 Get data from openml

In [5]:
data = oml.datasets.get_dataset(1480)
Xy = data.get_data()
Xy.shape

(583, 11)

In [6]:
discover_type_heuristic(Xy)

['int64',
 'bool',
 'float64',
 'float64',
 'int64',
 'int64',
 'int64',
 'float64',
 'float64',
 'float64',
 'bool']

## 2. Bayesian Method
To apply bayesian methos, dataset needs to be parsed into tabular numeric form beforehand.

### Get data from openml

In [172]:
data = oml.datasets.get_dataset(40498)
Xy = data.get_data()
Xy.shape

(4898, 12)

### Generate mat file

In [176]:
def generate_mat(data):
    simple_types = discover_type_heuristic(data)
    # map simple types to meta types
    # 1: real (w positive: all real | positive | interval)
    # 2: real (w/o positive: all real | interval)
    # 3: binary data
    # 4: discrete (non-binary: categorical | ordinal | count)
    # note: for now, the implemented bayesian method version by isabel can distinguish between real, postive real, categorical and count
    # the binary data should be mapped to meta type 4 discrete instead of meta type 3 due to the limited implemented version. This may change
    # if the extended version has been implemented by isabel.
    meta_types = [] 
    for i in range(len(simple_types)):
#         print(simple_types[i])
        if simple_types[i] == "bool":
            meta_types.append(4) # may change in the future
        elif simple_types[i] == "int64" or simple_types[i] == "float64":
            if (len(np.unique(data[:,i])) < 0.05 * len(data[:,i]) and len(np.unique(data[:,i])) < 50):
                meta_types.append(4)
            else:
                if (data[:,i] > 0).all():
                    meta_types.append(1)
                else:
                    meta_types.append(2)
    discrete_cardinality = [] # number of unique for discrete feature, 1 for others
    for i in range(len(meta_types)):
        if (meta_types[i] == 4):
            discrete_cardinality.append(len(np.unique(data[:,i])) + 50) # +50 to be on the safe side, otherwise one hot bug
        else:
            discrete_cardinality.append(1)
    data_dict = {'X': data,
                 'T': np.asarray(meta_types),
                 'R': np.asarray(discrete_cardinality)}
    print(data_dict)
    savemat('bayesian/data/temp/data.mat', data_dict, oned_as='row')

In [174]:
generate_mat(Xy)

{'X': array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  3.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  3.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  3.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  3.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  4.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  3.  ]], dtype=float32), 'T': array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7])}


### Call bayesian model

In [10]:
from bayesian.bin import abda

D:\Programfiles\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [165]:
weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)


Loaded bayesian/data/temp/data.mat
	with shape: (4898, 12)
	meta-types:[1 1 2 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [ 1  1  1  1  1  1  1  1  1  1  1 17]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 3.79999995, 14.19999981]), array([0.08      , 1.10000002]), array([0.        , 1.65999997]), array([ 0.60000002, 65.80000305]), array([0.009     , 0.34599999]), array([  2., 289.]), array([  9., 440.]), array([0.98711002, 1.03898001]), array([2.72000003, 3.81999993]), array([0.22      , 1.08000004]), array([ 8.        , 14.19999981]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<c

{'params': {'alpha': 87.14093462580814, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46E10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46278>}
gamma fit params (9.49794438454372, 0, 0.02980241542636818)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9B46588>}
{'params': {'alpha': 9.49794438454372, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46C18>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46E48>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9B46A58>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B469B0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algori

 4.50247636e-11])) + 0.3333333333333333*(Geometric(V11|p=0.5)) + 0.3333333333333333*(Poisson(V11|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Done iteration 1/1 in 0.2286798173072384


------------------
Done in 0.23499038768932223
Dumped id to ./exp/temp/data.mat\20181028-190728\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-190728\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-190728


In [166]:
weights

{0: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.0013964201090063983,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9986035798909936},
 1: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.24598741263987353,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7540125873601267},
 2: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.17914043194203624,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.8208595680579639},
 3: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.21018199626681527,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7898180037331847},
 4: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.18954843188562825,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.8104515681143718},
 5: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.0007417579232631625,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.999258242076737},
 6: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.0006899142006242295,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9993100857993759},
 7: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.4060452747470379,
  <Type.POSITIVE: (3, <MetaTy

## 3. Evaluation
We evaluate the bayesian method on the 30 most runs openml datasets. The ids of the 30 openml datasets are as follows: <br>
31, 1464, 334, 50, 333, 1504, 1494, 3, 1510, 1489, 37, 1479, 1487, 1063, 1471, 1467, 44, 1067, 1480, 1068, 1492, 1493, 1491, 1050, 1462, 1046, 1049, 335, 

In [53]:
def get_statistical_type(weights):
    """
    Retrieve the key with the higher value from 'weights'
    The retrieved key is the statisical type of the corresponding feature
    """
    statistical_types = []
    for i in range(len(weights)):
#         print(max(weights[i], key=weights[i].get))
        statistical_types.append(str(max(weights[i], key=weights[i].get)))
    return statistical_types

In [54]:
def statistical_type_accurracy(ground_truth, statistical_types):
    count = 0 # number of correct predictions
    for i in range(len(ground_truth)):
        if ground_truth[i] == statistical_types[i]:
            count += 1
    return count / len(ground_truth)

In [82]:
def evaluate_bayesian(dataset_id, ground_truth):
    data = oml.datasets.get_dataset(dataset_id)
    Xy = data.get_data()
    Xy.shape
    generate_mat(Xy)
    weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)
    statistical_types = get_statistical_type(weights)
    print(statistical_types)
    accuracy = statistical_type_accurracy(ground_truth, statistical_types)
    return accuracy

In [194]:
# 31
ground_truth = ['Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.POSITIVE',
                'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGOTICAL', 'Type.COUNT', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(31, ground_truth)

{'X': array([[ 0.,  6.,  4., ...,  1.,  0.,  0.],
       [ 1., 48.,  2., ...,  0.,  0.,  1.],
       [ 3., 12.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 3., 12.,  2., ...,  0.,  0.,  0.],
       [ 0., 45.,  2., ...,  1.,  0.,  1.],
       [ 1., 45.,  4., ...,  0.,  0.,  0.]], dtype=float32), 'T': array([4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([54, 83, 55, 60,  1, 55, 55, 54, 54, 53, 54, 54,  1, 53, 53, 54, 54,
       52, 52, 52, 52])}

Loaded bayesian/data/temp/data.mat
	with shape: (1000, 21)
	meta-types:[4 4 4 4 1 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [54 83 55 60  1 55 55 54 54 53 54 54  1 53 53 54 54 52 52 52 52]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.REAL: 1>

       1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE98E4978>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE98E4080>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98E4A20>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.93423595e-10, 1.93423595e-10, 1.93423595e-10, 7.73694398e-03,
       1.93423614e-03, 8.51063818e-02, 9.67117992e-03, 9.67117992e-03,
       4.06189551e-02, 2.90135394e-02, 1.35396518e-02, 1.89555123e-01,
       1.93423614e-03, 3.86847208e-03, 6.18955505e-02, 3.86847208e-03,
       1.93423595e-10, 1.04448741e-01, 1.93423595e-10, 9.67117992e-03,

       1.93423596e-10, 1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9931160>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9931F98>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9931320>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.21856865e-01, 2.14700191e-01, 1.56673113e-01, 5.06769821e-01,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,



1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.88007731e-01, 3.53965180e-01, 4.83558991e-02, 9.67117998e-03,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596

       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9B46780>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9B46A90>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46080>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.93995856e-01, 3.02277430e-01, 5.38302273e-02, 3.49896477e-01,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 

       2.07039335e-10, 2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9B461D0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9B46CC0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B460F0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([4.55486539e-02, 2.44306416e-01, 4.12008277e-01, 1.96687369e-01,
       1.01449274e-01, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-1

       2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9F616D8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9F611D0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9F614A8>}
gamma fit params (63.21435231509227, 0, 0.4195852453189975)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9689748>}
{'params': {'alpha': 63.21435231509227, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689E80>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689898>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': ar

 0. 0. 0. 0.]
{'params': {'p': array([6.85300200e-01, 3.14699790e-01, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posterior

       6.99300673e-10, 6.99300673e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBED51E4A8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBED51EA58>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED51E9E8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': array([3.91608378e-01, 6.99300673e-10, 6.08391587e-01, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-

 2.07039335e-10 2.07039335e-10 2.07039335e-10 2.07039335e-10])) + 0.3333333333333333*(Geometric(V20|p=0.5)) + 0.3333333333333333*(Poisson(V20|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Done iteration 1/1 in 0.13341689003209467


------------------
Done in 0.1383098914011498
Dumped id to ./exp/temp/data.mat\20181028-194900\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-194900\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-194900
['Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL']


In [195]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.47619047619047616


In [188]:
# 334
ground_truth = ['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(334, ground_truth)

{'X': array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 2., 2., ..., 2., 2., 1.],
       [0., 2., 2., ..., 2., 3., 0.],
       [0., 2., 2., ..., 2., 3., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4]), 'R': array([52, 53, 53, 52, 53, 54, 52])}

Loaded bayesian/data/temp/data.mat
	with shape: (601, 7)
	meta-types:[4 4 4 4 4 4 4]
	maximal discrete cardinality: [52 53 53 52 53 54 52]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51], dtype=int64), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       

{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689D68>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': array([2.86407760e-01, 3.73786399e-01, 3.39805817e-01, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.85436881e-10, 4.85436881e-10,
       4.85436881e-10, 4.85436881e-10, 4.8543688

       2.53164554e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9689780>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9689080>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE96896D8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': array([3.54430375e-01, 3.29113920e-01, 3.16455692e-01, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-10, 2.53164554e-10, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-10, 2.53164554e-10, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-10, 2.53164554e-10, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-10, 2.53164554e-10, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-10, 2.53164554e-10, 2.53164554e-10,
       2.53164554e-10, 2.53164554e-

 2.53164554e-10 2.53164554e-10 2.53164554e-10 2.53164554e-10])) + 0.3333333333333333*(Geometric(V6|p=0.5)) + 0.3333333333333333*(Poisson(V6|mean=2))))))
[0, 1, 2, 3, 4, 5, 6]
Done iteration 1/1 in 0.03930230813784874


------------------
Done in 0.04540326305323106
Dumped id to ./exp/temp/data.mat\20181028-192350\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-192350\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-192350
['Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT']


In [189]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.7142857142857143


In [179]:
# 1510
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1510, ground_truth)

{'X': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 4.601e-01, 1.189e-01,
        1.000e+00],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 2.750e-01, 8.902e-02,
        1.000e+00],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 3.613e-01, 8.758e-02,
        1.000e+00],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 2.218e-01, 7.820e-02,
        1.000e+00],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 4.087e-01, 1.240e-01,
        1.000e+00],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 2.871e-01, 7.039e-02,
        0.000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 4]), 'R': array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 52])}

Loaded bayesian/data/temp/data.mat
	with shape: (569, 31)
	meta-types:[1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 4]
	maximal discrete cardinality: [ 1  1  1  1

{'params': {'alpha': 52.41211939552996, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0D4A8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0DDD8>}
gamma fit params (69.959569224947, 0, 0.0009124183519733392)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9D0DC18>}
{'params': {'alpha': 69.959569224947, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0D4E0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0D9E8>}
gamma fit params (3.9164005526527883, 0, 0.14765090446519819)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9D0DAC8>}
{'params': {'alpha': 3.9164005526527883, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obje

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689438>}
gamma fit params (15.31566001428902, 0, 0.011603921876071932)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9689198>}
{'params': {'alpha': 15.31566001428902, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE96892E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689BE0>}
gamma fit params (21.48990769214565, 0, 0.014842222109061783)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9689B00>}
{'params': {'alpha': 21.48990769214565, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689E10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE96897B8>}
ga

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBED51E278>}
{'params': {'alpha': 7.595855353877886, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED51EB38>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED51E080>}
gamma fit params (6.260240000357926, 0, 3.345015242480662)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBED51EA58>}
{'params': {'alpha': 6.260240000357926, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED51E860>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED51E978>}
gamma fit params (6.293524158461141, 0, 0.0011342489168239965)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object 

       3.02114799e-10, 3.02114799e-10, 3.02114799e-10, 3.02114799e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE967A080>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE967A588>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE967AF28>}
True True True
True True True
(0.4182776801405975*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=25.998594799117946;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=29.039720693365776;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=24.94717198627268;beta=1)) + 0.3333333333333333*(Exponential(V2|

 3.02114799e-10 3.02114799e-10 3.02114799e-10 3.02114799e-10])) + 0.3333333333333333*(Geometric(V30|p=0.5)) + 0.3333333333333333*(Poisson(V30|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Done iteration 1/1 in 0.21187149900652003


------------------
Done in 0.2182625996265415
Dumped id to ./exp/temp/data.mat\20181028-191537\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-191537\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-191537
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', '

In [180]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [181]:
# 1489
ground_truth = ['Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1489, ground_truth)

{'X': array([[ 0.489927, -0.451528, -1.04799 , -0.598693, -0.020418,  0.      ],
       [-0.641265,  0.109245,  0.29213 , -0.916804,  0.240223,  0.      ],
       [ 0.870593, -0.459862,  0.578159,  0.806634,  0.835248,  0.      ],
       ...,
       [ 0.246882, -0.793228,  1.190101,  1.423194, -1.303036,  1.      ],
       [-0.778907, -0.383111,  1.727029, -1.432389, -1.208085,  0.      ],
       [-0.794604, -0.640053,  0.632221,  0.72028 , -1.231182,  1.      ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 2, 4]), 'R': array([ 1,  1,  1,  1,  1, 52])}

Loaded bayesian/data/temp/data.mat
	with shape: (5404, 6)
	meta-types:[2 2 2 2 2 4]
	maximal discrete cardinality: [ 1  1  1  1  1 52]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([-2.933743 ,  3.8285749]), array([-3.03870893,  3.66476297]), array([-2.79586911,  2.63086796]), array([-2.48534012,  3.04694796]), array([-2.36746097,  4

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9CDCB00>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9D0D710>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0D940>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9D0DD68>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9A942B0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9A94CC0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9A94E10>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9A94048>}
{'params': {'alpha

In [182]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5


In [183]:
# 1479
ground_truth = []
for i in range(100):
    ground_truth.append('Type.POSITIVE')
ground_truth.append('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

       51], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE2158198>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE2158710>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE5D04128>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'p

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9240>}
gamma fit params (1.7217319919687364, 0, 3.256745266208983)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9B38>}
{'params': {'alpha': 1.7217319919687364, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E97B8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9630>}
gamma fit params (1.7236174215686124, 0, 3.2694508191991907)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9A90>}
{'params': {'alpha': 1.7236174215686124, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E98D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9F60>}
gam

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9EB99E8>}
{'params': {'alpha': 1.7290673410065707, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9EB92E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9EB9C50>}
gamma fit params (1.7104641158781813, 0, 3.312131687401482)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9EB9518>}
{'params': {'alpha': 1.7104641158781813, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9EB9470>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9EB94A8>}
gamma fit params (1.7159805690787073, 0, 3.2962566122679218)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object

{'params': {'alpha': 1.7657624529922589, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAC18>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FA550>}
gamma fit params (1.7465505733488662, 0, 3.219387981609981)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE98FA048>}
{'params': {'alpha': 1.7465505733488662, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAE10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAC88>}
gamma fit params (1.7418494713027437, 0, 3.224357564176131)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE98FA7B8>}
{'params': {'alpha': 1.7418494713027437, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obj

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBECF78358>}
gamma fit params (1.7790793827106595, 0, 3.1554264618655368)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBECF785C0>}
{'params': {'alpha': 1.7790793827106595, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBECF789B0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBECF786A0>}
gamma fit params (1.8056958209536014, 0, 3.075612851874063)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBECF78A90>}
{'params': {'alpha': 1.8056958209536014, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBECF78470>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBECF780B8>}
gam

 1.65016500e-10 1.65016500e-10 1.65016500e-10 1.65016500e-10])) + 0.3333333333333333*(Geometric(V100|p=0.5)) + 0.3333333333333333*(Poisson(V100|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.6532146959789316


------------------
Done in 1.7364537484063476
Dumped id to ./exp/temp/data.mat\20181028-191733\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-191733\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-191733
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.PO

In [184]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9900990099009901


In [115]:
# 1487
ground_truth = []
for i in range(73):
    ground_truth.append('Type.POSITIVE')
for i in [54, 55, 60, 62, 65, 70]:
    ground_truth[i] = 'Type.REAL'
ground_truth[72] = ('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE2158198>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE2158710>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE5D04128>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'alpha': 1.7917450699943174, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9BA07F0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9BA0278>}
gamma fit params (1.7806787234431918, 0, 58.509740076549804)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9C00518>}
{'params': {'alpha': 1.7806787234431918, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9FA78D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9FA7080>}
gamma fit params (1.8087179444833978, 0, 57.68304631811555)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9FA7FD0>}
{'params': {'alpha': 1.8087179444833978, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma ob

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9358>}
gamma fit params (1.7776011109010725, 0, 58.92994406822095)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9978>}
{'params': {'alpha': 1.7776011109010725, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E93C8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9B00>}
gamma fit params (1.7646087863499302, 0, 59.71978961279681)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9748>}
{'params': {'alpha': 1.7646087863499302, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9908>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E98D0>}
gamm

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE995EBE0>}
{'params': {'alpha': 1.762958122630515, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE995EFD0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE995E5F8>}
gamma fit params (1.7776716177560679, 0, 58.647022644643336)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE995E630>}
{'params': {'alpha': 1.7776716177560679, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE995EC88>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE995E8D0>}
gamma fit params (1.804586229637459, 0, 57.63433705304342)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

(0.5*(((0.5066006600660066*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.6545988685199116;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.6570433180904642;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.6414960718966134;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.656301074955083;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.6658529706781784;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUncons

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.436130443192269


------------------
Done in 1.4589497120250599
Dumped id to ./exp/temp/data.mat\20181028-181948\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-181948\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-181948
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIV

In [116]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9041095890410958


In [185]:
# data = oml.datasets.get_dataset(31)
# Xy = data.get_data()
# Xy.shape
# generate_mat(Xy)
# weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
#      args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
#      args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
#      args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
#      args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
#      args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)

{'X': array([[ 0.,  6.,  4., ...,  1.,  0.,  0.],
       [ 1., 48.,  2., ...,  0.,  0.,  1.],
       [ 3., 12.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 3., 12.,  2., ...,  0.,  0.,  0.],
       [ 0., 45.,  2., ...,  1.,  0.,  1.],
       [ 1., 45.,  4., ...,  0.,  0.,  0.]], dtype=float32), 'T': array([4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([54, 83, 55, 60,  1, 55, 55, 54, 54, 53, 54, 54,  1, 53, 53, 54, 54,
       52, 52, 52, 52])}

Loaded bayesian/data/temp/data.mat
	with shape: (1000, 21)
	meta-types:[4 4 4 4 1 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [54 83 55 60  1 55 55 54 54 53 54 54  1 53 53 54 54 52 52 52 52]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.REAL: 1>

       1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9B809E8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9B80128>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B80E80>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.93423595e-10, 1.93423595e-10, 1.93423595e-10, 7.73694398e-03,
       1.93423614e-03, 8.51063818e-02, 9.67117992e-03, 9.67117992e-03,
       4.06189551e-02, 2.90135394e-02, 1.35396518e-02, 1.89555123e-01,
       1.93423614e-03, 3.86847208e-03, 6.18955505e-02, 3.86847208e-03,
       1.93423595e-10, 1.04448741e-01, 1.93423595e-10, 9.67117992e-03,

       1.93423596e-10, 1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE98E49B0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE98E4438>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98E4E48>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.21856865e-01, 2.14700191e-01, 1.56673113e-01, 5.06769821e-01,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,



1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.88007731e-01, 3.53965180e-01, 4.83558991e-02, 9.67117998e-03,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10,
       1.93423596e-10, 1.93423596

       1.93423596e-10, 1.93423596e-10, 1.93423596e-10, 1.93423596e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9B464A8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9B46C18>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9B46278>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.93995856e-01, 3.02277430e-01, 5.38302273e-02, 3.49896477e-01,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 

       2.07039335e-10, 2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9F619B0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9F61F98>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9F61048>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([4.55486539e-02, 2.44306416e-01, 4.12008277e-01, 1.96687369e-01,
       1.01449274e-01, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-1

       2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9689828>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9689160>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689208>}
gamma fit params (63.21435231509227, 0, 0.4195852453189975)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9F61D68>}
{'params': {'alpha': 63.21435231509227, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689780>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9689470>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': ar

 0. 0. 0. 0.]
{'params': {'p': array([6.85300200e-01, 3.14699790e-01, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10,
       2.07039335e-10, 2.07039335e-10, 2.07039335e-10, 2.07039335e-10])}, 'prior': <bayesian.spn.algorithms.Posterior

       6.99300673e-10, 6.99300673e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x000000CBE9D15FD0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x000000CBE9D15978>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBED8026D8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
{'params': {'p': array([3.91608378e-01, 6.99300673e-10, 6.08391587e-01, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-10, 6.99300673e-10, 6.99300673e-10, 6.99300673e-10,
       6.99300673e-

 2.07039335e-10 2.07039335e-10 2.07039335e-10 2.07039335e-10])) + 0.3333333333333333*(Geometric(V20|p=0.5)) + 0.3333333333333333*(Poisson(V20|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Done iteration 1/1 in 0.1500973073689238


------------------
Done in 0.15503886373426212
Dumped id to ./exp/temp/data.mat\20181028-191905\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-191905\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-191905
